In [1]:
# coding: utf-8

get_ipython().system(' pip install --default-timeout=180 boto')
get_ipython().system(' pip install --default-timeout=180 pymongo')
get_ipython().system(' pip install --default-timeout=180 pymssql')
get_ipython().system(' pip install --default-timeout=180 psycopg2')
get_ipython().system(' pip install --default-timeout=180 WISE-PaaS-DataHub-Edge-Python-SDK')
get_ipython().system(' pip install --default-timeout=180 func_timeout')


Looking in indexes: http://10.100.0.6:8080
Looking in indexes: http://10.100.0.6:8080
Looking in indexes: http://10.100.0.6:8080
Looking in indexes: http://10.100.0.6:8080
Looking in indexes: http://10.100.0.6:8080
Looking in indexes: http://10.100.0.6:8080


In [2]:
import boto
import boto.s3.connection
from boto.s3.key import Key 

import re
import os
import requests
import urllib.parse
import time
import json
import datetime
import sqlite3
import struct     
import tempfile
import csv
import binascii
import sys
import inspect
import pymssql

import numpy as np
import pandas as pd

from func_timeout import func_timeout, FunctionTimedOut
from pymongo import MongoClient, DESCENDING

#PostgreSQL access
import psycopg2
import pandas.io.sql as sqlio

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
from wisepaasdatahubedgesdk.EdgeAgent import EdgeAgent
from wisepaasdatahubedgesdk.Model.Edge import EdgeAgentOptions, MQTTOptions, DCCSOptions, NodeConfig, EdgeConfig
from wisepaasdatahubedgesdk.Model.Edge import DeviceConfig, AnalogTagConfig, EdgeData, EdgeTag, TextTagConfig
import wisepaasdatahubedgesdk.Common.Constants as constant

In [4]:
# scada_info = \
# [{'Scada project': 'Y4_FOMOS_1HSM',
# 'Credential Key': '50099a2bb15fa8eb93b05261ef3d6dm5',
# 'DCCS API Url': 'https://api-dccs.fomos.csc.com.tw/'}
# ,{
# 'Scada project': 'Y4_FOMOS_1HSM',
# 'Credential Key': 'a4e194df77fdbbf2ccf56a9f6632bfwk',
# 'DCCS API Url': 'https://api-dccs.fomos.csc.com.tw/'}
# ,{
# 'Scada project': 'Y4_FOMOS_1HSM',
# 'Credential Key': '3fcbd86c8edd234bfc1129629f36aetw',
# 'DCCS API Url': 'https://api-dccs.fomos.csc.com.tw/'}
# ,{
# 'Scada project': 'Y4_FOMOS_1HSM',
# 'Scada name': '122',
# 'Credential Key': '9218fd01dbe29dabab2002fe6f34e4ie',
# 'DCCS API Url': 'https://api-dccs.fomos.csc.com.tw/'}
# ,{
# 'Scada project': 'Y4_FOMOS_1HSM',
# 'Scada name': 'vpodpro',
# 'Credential Key': '74932c8f872342d643cb73d890a5aebe',
# 'DCCS API Url': 'https://api-dccs.fomos.csc.com.tw/'}]
# scada_info = pd.DataFrame(scada_info)

In [5]:
# if 'dccs_key' not in os.environ:
#      os.environ['dccs_key'] = '50099a2bb15fa8eb93b05261ef3d6dm5'

if 'scada_id' not in os.environ:
     os.environ['scada_id'] = '14d267b3-b03c-48d3-9b54-5ca79d0c457b'

if 'scada_name' not in os.environ:
    os.environ['scada_name'] = '111'
    
if 'from' not in os.environ:
    os.environ['from'] = '2020-12-01'
    
if 'to' not in os.environ:
    today_twd = datetime.datetime.now() + datetime.timedelta(hours=8)
    os.environ['to'] = today_twd.strftime('%Y-%m-%d')
    
if 'points' not in os.environ:
    os.environ['points'] = '2'

In [6]:
# True if write to SCADA
bWrite2Scada = True

# True if write to SCADA and Mongodb
bForceExtract = False

In [7]:

mgdb_host = '10.100.10.1'
mgdb_port = '27017'
mgdb_username = '0c1e58e3-8643-4ff3-8633-7820f10f4902'
mgdb_password = 'SPRXaEL2RIIeve2lsHV9oAjCc'
mgdb_database = 'd21d5987-3b65-4fae-9b02-79f72b39b735'
mgdb_collection = 'y4_features'

In [8]:
# yesterday = datetime.datetime.today() - datetime.timedelta(days=1)

# year = yesterday.strftime('%Y')
# month = yesterday.strftime('%m')
# day = yesterday.strftime('%d')

In [9]:
ACCESS_KEY = 't7user1'
SECRET_KEY = 'Bhw0MdOHfFL9h03u3epl4AoLxl/sOu0UvELUBL1h'
HOST = 'object.csc.com.tw'    #172.16.247.234
PORT = 9020
BUCKET_NAME = 'Y4_HSM'

In [10]:
PG_IP = "192.168.123.238"
PG_USER = "6e6b8fc2-ea3d-412e-9806-692a4aea5c0e"
PG_PASS = "huu1enrd9rrsptr86kvapqk4pe"
PG_DB = "214c2b8f-c62b-4133-82a8-93eb2dc59277"

In [11]:
# datahub
DHB_MQTT_HOST = '10.10.0.14'
DHB_MQTT_PORT = 1883
DHB_MQTT_USER = "6d657c0a-7fc9-44be-b220-819efb23fbdb:403e3cd7-92ef-457b-9309-64970864bde5"
DHB_MQTT_PASS = "u3s9tqk0bvn9sklt6dr8j1jpe9"

In [12]:
s3_connection = boto.connect_s3(
                   aws_access_key_id = ACCESS_KEY,
                   aws_secret_access_key = SECRET_KEY,
                   host = HOST,
                   port = PORT,
                   is_secure=False,               # uncomment if you are not using ssl
                   calling_format = boto.s3.connection.OrdinaryCallingFormat(),
                 )
bucket = s3_connection.get_bucket(BUCKET_NAME, validate=False)

In [13]:
conn = psycopg2.connect(
    host = PG_IP,
    database = PG_DB,
    user = PG_USER,
    password = PG_PASS)

sql = r'SELECT * FROM "CSC_FOMOS"."Y4_Channel_List";'
tag_list_pd = sqlio.read_sql_query(sql, conn)
conn.close()

In [14]:
scada_project = 'Y4_FOMOS_1HSM'
scada_name = os.environ['scada_name']
scada_id = os.environ['scada_id']

device_name_list = list(tag_list_pd[tag_list_pd['smartboxID'] == scada_name]['channel_name'])
device_id_list = list(tag_list_pd[tag_list_pd['smartboxID'] == scada_name]['channelID'])

# for development
device_range = int(os.environ['points'])
device_name_list = device_name_list[0:device_range]
device_id_list = device_id_list[0:device_range]

In [15]:
# print('scada_project=',scada_project, '\n', 
#       'scada_name=',scada_name, '\n',
#      'device_name=',device_name_list[0],'\n',
#      'device_id=',device_id_list[0],'\n',
#      'scada_id=',scada_id)

In [16]:

for device_id in device_id_list:
    
    options = EdgeAgentOptions(
        reconnectInterval = 1, # MQTT reconnect interval seconds
        nodeId = scada_id, # Getting from SCADA portal
        deviceId = device_id, # If type is Device, DeviceId must be filled
        type = constant.EdgeType['Gateway'], # Choice your edge is Gateway or Device, Default is Gateway
        heartbeat = 60, # Default is 60 seconds
        dataRecover = False, # Need to recover data or not when disconnected
        connectType = constant.ConnectType['MQTT'], # Connection type (DCCS, MQTT), default is DCCS
        
        MQTT = MQTTOptions( # If connectType is MQTT, must fill this options
            hostName = DHB_MQTT_HOST,
            port = DHB_MQTT_PORT,
            userName = DHB_MQTT_USER,
            password = DHB_MQTT_PASS,
            protocalType = constant.Protocol['TCP'] # MQTT protocal (TCP, Websocket), default is TCP
        )
    )

    edgeAgent = EdgeAgent( options = options )
    edgeAgent.connect()
    #Add 2020/6/13
    time.sleep(3)

Connected OK Returned code= 0
subscribe /wisepaas/scada/14d267b3-b03c-48d3-9b54-5ca79d0c457b/ack successfully
subscribe /wisepaas/scada/14d267b3-b03c-48d3-9b54-5ca79d0c457b/cmd successfully
subscribe successfully
subscribe successfully
Connected OK Returned code= 0
subscribe /wisepaas/scada/14d267b3-b03c-48d3-9b54-5ca79d0c457b/ack successfully
subscribe /wisepaas/scada/14d267b3-b03c-48d3-9b54-5ca79d0c457b/cmd successfully
subscribe successfully
subscribe successfully


In [17]:
def write_binfile (new_filename, signal, dtype='f'):
    
    # vPorPRO_only
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
    
    float_array = signal.astype(dtype).flatten()
    data = struct.pack(dtype*float_array.shape[0], *float_array)

    f = open(new_filename.encode('utf-8'),'wb')
    int_array = np.array(signal.shape).reshape(1,-1).flatten()
    siz_data = struct.pack('i'*int_array.shape[0], *int_array)
    f.write(siz_data)
    f.write(data)    
    f.close()

In [18]:
def read_binfile (filename, dtype='f'):
    
    # vPorPRO_only
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
    #example: 
    # signal = read_bin('Raw Data-1-1Y510110100-00-03-15.bin'); 
    # import matplotlib.pyplot as plt; plt.plot(signal.T); 
    # plt.show()
    # import binascii
    # import numpy as np

    def hexint(b, bReverse=True): return int(binascii.hexlify(b[::-1]), 16) if bReverse else int(binascii.hexlify(b), 16)    
    bytes_read = open(filename, "rb").read()
    
    #read matrix size, 4-byte unit
    size = [hexint(bytes_read[(i*4):((i+1)*4)]) for i in range(2)]
    # print(size)
    
    # import struct
    signal = [struct.unpack(dtype,bytes_read[(i*4):((i+1)*4)]) for i in range(2, 2+size[0]*size[1])]
    
    return np.array(signal).reshape(size)
    

In [19]:
def s3_upload (bucket_, key, testfile): 
    
    # vPorPRO_only
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)

    def percent_cb(complete, total):
        sys.stdout.write('.')
        sys.stdout.flush()
    
    k = Key(bucket_)
    k.key = key
    
    print(testfile.encode('utf-8'))
    print(type(testfile))
    
    k.set_contents_from_filename(testfile.decode('utf-8'), cb=percent_cb, num_cb=10)
    

In [20]:
def retrieve_bin_from_1db (filename, bucket, mv2g=10, one_unit=8, fs=25600):
    
    # vPorPRO_only
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)

    
#     if opts is not None:
#         smartboxID = opts['smartboxID']
#         ScadaID = opts['ScadaID']
#         scada_project = opts['scada_project']
#         scada_name = opts['scada_name']
#         scada_id = opts['scada_id']
#         device_name = opts['device_name']
#         device_id = opts['device_id'] 
#         bucket = opts['bucket']
#         tag_list_pd = opts['tag_list_pd']
#     print('*****{} connect to the DB.'.format(datetime.datetime.strftime(\
#                     datetime.datetime.now(),'%Y-%m-%d %H:%M:%S.%f')))
    bucket = s3_conn()
    sql_conn = sqlite3.connect(filename.decode('utf-8'))
    
    #check index for debugging purposes
    if True:
#         print('*****{} check index.'.format(datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S.%f')))
        indices = sql_conn.execute("PRAGMA INDEX_LIST('records')")
        point_record_colnames = list(map(lambda x: x[0], indices.description))
        point_record_row = indices.fetchone()
        
        if point_record_row is None: #create indices
#             print('*****{} create index.'.format(datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S.%f')))
            sql_conn.execute('CREATE INDEX "" ON "records" ("point_id" ASC)')
        else:
            indices = pd.DataFrame(list(point_record_row),index=point_record_colnames)
        
    #point_record
#     print('*****{} retrieve data.'.format(datetime.datetime.strftime(\
#                     datetime.datetime.now(),'%Y-%m-%d %H:%M:%S.%f')))
    rows = sql_conn.execute('select records.point_id,measure_time,vibration,bearing,temperature,\
                         raw_data,points.name from records LEFT JOIN measures on \
                         records.point_id = measures.point_id LEFT JOIN points on \
                         points.id = measures.point_id LEFT JOIN machines on \
                         points.machine_id = machines.id')

    point_record_colnames = list(map(lambda x: x[0], rows.description))
        
    point_record_row = rows.fetchall()
    
    if len(point_record_row)>0:
        rtb = pd.DataFrame(list(point_record_row),columns=point_record_colnames)
    else:
        rtb = pd.DataFrame([list(point_record_row)],columns=point_record_colnames)
        
    dtt = [datetime.datetime.fromtimestamp(r) for r in rtb['measure_time']]
    rtb['timestamp'] = rtb['measure_time']
    rtb['measure_time'] = dtt
    
    sql_conn.close()

#     tmpfile = tempfile.TemporaryFile().name.split('\\')[-1] + '.bin'  
    
    for index, row in rtb.iterrows():
        bytes_read = row['raw_data']
        size = int(len(bytes_read) / one_unit)
        signal_a = [struct.unpack('d',bytes_read[(i*one_unit):((i+1)*one_unit)]) for i in range(size)]
        signal_a = np.array(signal_a).reshape(size) * mv2g
        #[re.findall('[0-9]+',row['name'])[0] for index, row in rtb.iterrows()]
        
        rowID = re.findall('[0-9]+',row['name'])[0]
        channel_name = '#{}內冷式ROT Roller WS_vpod'.format(rowID)
        tag_list_row = tag_list_pd[tag_list_pd['channel_name']==channel_name]
        device = tag_list_row['channelID'].iloc[0]
        year = "{:04d}".format(row['measure_time'].year)
        month = "{:02d}".format(row['measure_time'].month)
        day = "{:02d}".format(row['measure_time'].day)
        hour = "{:02d}".format(row['measure_time'].hour)
        minute = "{:02d}".format(row['measure_time'].minute)
        second = "{:02d}".format(row['measure_time'].second)
        
        prefix = '#1HSM/ROT/vPodPRO/{}/{}/{}/{}/'.format(channel_name,year,month,day)
        S3_key_name = prefix + 'Raw Data-{}-{}-{}-{}_{}.bin'.format(channel_name,hour,minute,second,fs)
        
        #print('tmpfile=', S3_key_name.split('/')[-1])
        tmpfile = S3_key_name.split('/')[-1]
        #write_binfile('tmp.bin',signal_a.reshape(1,-1))
        
        write_binfile(tmpfile, signal_a.reshape(1,-1))
        #read signal_a2=read_binfile('tmp.bin').flatten(); np.max(np.abs(signal_a-signal_a2))
        
        # TODO: ask Dr Ho the reason of this?
        #s3_upload(bucket,S3_key_name,tmpfile)
        
        #extract features
#         opts['device_name'] = channel_name
#         opts['device_id'] = device
        
        #2020-09-24
#         print('retrieve_bin_from_1db',scada_id,opts['device_name'],opts['device_id'])
        extract_features(prefix=prefix,
                         fs=fs,
                         device_name=device,
                         vibration=row['vibration'],
                         bearing=row['bearing'],
                         temperature=row['temperature'],
                         opts=None)
        
    #delete tmp files
    os.remove(tmpfile.encode('utf-8'))  
    
    pass
    

In [21]:
def retrieve_bin_from_dbs (src='RAW_DB', dest='RAW_DB_BACK', opts=None):
    
    # vPorPRO_only
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
    
#     if opts is not None:
#         smartboxID = opts['smartboxID']
#         ScadaID = opts['ScadaID']
#         scada_project = opts['scada_project']
#         scada_name = opts['scada_name']
#         scada_id = opts['scada_id']
#         device_name = opts['device_name']
#         device_id = opts['device_id'] 
#         bucket = opts['bucket']
    
    bucket = s3_conn()
    prefix_ = '#1HSM/ROT/vPodPRO/'+src+'/'
    back_prefix = '#1HSM/ROT/vPodPRO/'+dest+'/'
    db_keys = [key for key in bucket.list(prefix=prefix_) if len(re.findall('.db$',key.name))>0]
    
#     print(db_keys)
    
    #filename = 'tmpp.db'
#     filename = tempfile.TemporaryFile().name.split('\\')[-1] + '.db'
    for db_key in db_keys: 
        
#         print(str(db_key).split('/')[-1].split('>')[0])

        filename = str(db_key).split('/')[-1].split('>')[0].encode('utf-8')
    
        db_key.get_contents_to_filename(filename)
        retrieve_bin_from_1db(filename,bucket)
        dest_name = db_key.name.replace('RAW_DB',dest)
        db_key.copy(bucket,dest_name)
        db_key.delete()
        os.remove(filename)

In [22]:
def retrieve_ts (objs, 
                 filename,
                 type ='tdms'):    

    #print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
    
    year = int(objs.split('/')[-4])
    month = int(objs.split('/')[-3])
    day = int(objs.split('/')[-2])
    hour = int(filename.split('-')[-3])
    minute = int(filename.split('-')[-2])
    
    if type=='tdms':
        second = int(filename.split('-')[-1].split('.')[0])
        
    if type=='bin':
        second = int(filename.split('-')[-1].split('_')[0])        
        
    #ts = datetime.datetime(year, month, day, hour, minute, second).strftime('%s')
    ts = datetime.datetime(year, month, day, hour, minute, second).timestamp()
    
    return int(ts)

In [23]:
def insert_mgdb (data_dict):
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
    
    client = MongoClient('mongodb://%s:%s@%s/%s' % (mgdb_username, mgdb_password, mgdb_host, mgdb_database))

    db = client[mgdb_database]
    collection = db[mgdb_collection]    
    
    for i in data_dict:
        if "." in i:
            data_dict[i.replace(".", "$")] = data_dict.pop(i)
    
    #filter_dict = {'RMS_WDEC0': data_dict['RMS_WDEC0']}
    filter_dict = {"device" : data_dict["device"], "timestamp" : data_dict["timestamp"]}
    update_dict = {'$set': data_dict}
    
    doc = collection.update_one( filter_dict, update_dict,  upsert=True)
    #print(dir(doc))
    client.close()

In [24]:
def query_mgdb (date_from, date_to, device_id):
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)

    client = MongoClient('mongodb://%s:%s@%s/%s' % (mgdb_username, mgdb_password, mgdb_host, mgdb_database))
    
    gt = int(datetime.datetime.strptime(date_from + ' 0:0:0' , '%Y-%m-%d %H:%M:%S').timestamp())
    lt = int(datetime.datetime.strptime(date_to + ' 23:59:59', '%Y-%m-%d %H:%M:%S').timestamp())
    
    db = client[mgdb_database]
    collection = db[mgdb_collection]
    
    validation_df = pd.DataFrame(list(collection.find({ 
        '$and': [ 
            { 'timestamp': {'$gt':gt, '$lt': lt} }, 
            { 'device': device_id } ] })))
    
    # if MongoDB has no data, use the updated data, owen add this at 2020-05-29
    if len(validation_df) == 0:
        mgb_last_list = list(collection.find({ 'device':device_id }).sort('timestamp', DESCENDING).limit(10))
        
        if len(mgb_last_list)>0:
            print(datetime.datetime.now(), '| got 0 documents from mongodb, use old data:', mgb_last_list[0]['timestamp'])
            validation_df = pd.DataFrame(list(collection.find({ 'device':device_id }).sort('timestamp', DESCENDING).limit(10)))
            
        else:
            pass
    else:
        print(datetime.datetime.now(), '| more than one points in MongoDB')

    client.close()
    
    return validation_df

In [25]:
def query_mgdb_stime (dt, device_id):
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)

    client = MongoClient('mongodb://%s:%s@%s/%s' % (mgdb_username, mgdb_password, mgdb_host, mgdb_database))
    
    #dt = int(datetime.datetime.strptime(date_from + ' 0:0:0' , '%Y-%m-%d %H:%M:%S').timestamp())
    
    db = client[mgdb_database]
    collection = db[mgdb_collection]
    
    validation_df = pd.DataFrame(list(collection.find({ 
        '$and': [ 
            { 'timestamp': dt }, 
            { 'device': device_id } ] })))
    
    client.close()
    
    # if MongoDB has no data, return False
    if len(validation_df) == 0:
        return False
    else:
        return True
    

In [26]:
def query_mgdb_dt (date_from, date_to):
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)

    client = MongoClient('mongodb://%s:%s@%s/%s' % (mgdb_username, mgdb_password, mgdb_host, mgdb_database))
    
    gt = int(datetime.datetime.strptime(date_from + ' 0:0:0' , '%Y-%m-%d %H:%M:%S').timestamp())
    lt = int(datetime.datetime.strptime(date_to + ' 23:59:59', '%Y-%m-%d %H:%M:%S').timestamp())
    
    db = client[mgdb_database]
    collection = db[mgdb_collection]
    
    validation_df = pd.DataFrame(list(collection.find({ 
        '$and': [ 
            { 'timestamp': {'$gt': gt}}, 
            { 'timestamp': {'$lt': lt }} ]
    })))
    
    # if MongoDB has no data, use the updated data, owen add this at 2020-05-29
    if len(validation_df) == 0:
        mgb_last_list = list(collection.find({ 'device':device_id }).sort('timestamp', DESCENDING).limit(10))
        
        if len(mgb_last_list)>0:
            print(datetime.datetime.now(), '| got 0 documents from mongodb, use old data:', mgb_last_list[0]['timestamp'])
            validation_df = pd.DataFrame(list(collection.find({ 'device':device_id }).sort('timestamp', DESCENDING).limit(10)))
            
        else:
            pass
    else:
        print(datetime.datetime.now(), '| more than one points in MongoDB')

    client.close()
    
    return validation_df

In [27]:
WS_HOST = '172.16.247.48'
WS_PORT = '54321'

In [28]:
MSSQL_HOST = '10.20.246.188'
MSSQL_USER = 'dioc'
MSSQL_PASS = 'dioct16w68'
MSSQL_DB = 'FOMOS_AI_SERVER'

In [29]:
def query_features (filename, prefix, scada_name,fs=-1):
    
#     print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
#     import urllib.parse
#     import requests
#     import json
    
#     ip = '172.16.247.48'
#     port = '54321'
    
    filename = urllib.parse.unquote(filename)
    filename = filename.replace(' ', '%20')
    filename = filename.replace('#', '%23')
    
    prefix = prefix.replace('#', '%23')

    app_args = ''
    if False:
        app_args = '&host={}&port={}&bucket={}&accessKey={}&secretKey={}'.format(\
                    HOST,PORT,BUCKET_NAME,ACCESS_KEY,SECRET_KEY)
    
    if scada_name.lower()=='vpodpro':#not finished yet!
        app_args1 = '&SamplingRate={}'.format(fs)
        url = 'http://'+WS_HOST+':'+WS_PORT+'/ExtractFeatureS3BIN?key='+prefix+filename+app_args1
    else:
        url = 'http://'+WS_HOST+':'+WS_PORT+'/ExtractFeatureS3TDMS?key='+prefix+filename+app_args
    
    r = requests.get(url)
    data_json = json.loads(r.text)
    
    return data_json

In [30]:
def scada_client (data_dict):
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
#     from wisepaasdatahubedgesdk.EdgeAgent import EdgeAgent
#     from wisepaasdatahubedgesdk.Model.Edge import EdgeAgentOptions, MQTTOptions, DCCSOptions, NodeConfig, EdgeConfig
#     from wisepaasdatahubedgesdk.Model.Edge import DeviceConfig, AnalogTagConfig, EdgeData, EdgeTag, TextTagConfig
#     import wisepaasdatahubedgesdk.Common.Constants as constant
#     import datetime
#     import pandas as pd 
    
#     if opts is not None:
#         smartboxID = opts['smartboxID']
#         ScadaID = opts['ScadaID']
#         scada_project = opts['scada_project']
#         scada_name = opts['scada_name']
#         scada_id = opts['scada_id']
#         device_name = opts['device_name']
#         device_id = opts['device_id'] 
        
        #if device is None: device = device_id
#         date = opts['date']
#         AllFOMOSStationSensorInfo = opts['AllFOMOSStationSensorInfo']
#         FOMOSTsConstraint = opts['FOMOSTsConstraint']
#         edgeAgent = opts['edgeAgent']
#         bWrite2Scada = opts['bWrite2Scada']
        
    output_df = pd.DataFrame(list(data_dict.items()),columns = ['features','values']).T
    output_df.columns = output_df.iloc[0]
    output_df = output_df.drop(['features'])
    output_df = output_df.drop(columns=['type'])
    output_df = output_df.drop(columns=['device'])
    output_df = output_df.drop(columns=['timestamp'])
    output_df = output_df.drop(columns=['scada'])
    
    # print('scada session #1')
    
    #print(data_dict['device']

    
    ## session 2
    config = EdgeConfig()

    # print('scada session #2')
    nodeConfig = NodeConfig(#name = scada_name,
                              #description = 'Machine Name',
                              #description = scada_name,
                              #primaryIP = None,
                              #backupIP = None,
                              #primaryPort = None,
                              #backupPort = None,
                              nodeType = constant.EdgeType['Device']
                             )
    config.node = nodeConfig

    # print('scada session #3')
    deviceConfig = DeviceConfig(id = device_id,
                                name = device_name,
                                #comPortNumber = None,
                                deviceType = 'Device Type',
                                #description = 'w46 Device',
                                description = device_name
                                #ip = None,
                                #port = None
                               )
    config.node.deviceList.append(deviceConfig)

    # print('scada session #4')
    for feature in list(output_df):
    #     textTag = TextTagConfig(name = feature ,
    #                             description = feature ,
    #                             readOnly = True, 
    #                             arraySize = 0)
        analogTag = AnalogTagConfig(name = feature,
                                    description = feature,
                                    readOnly = False,
                                    arraySize = 0,
                                    spanHigh = 100000000000000000000000000000,
                                    spanLow = -100000000000,
                                    engineerUnit = '',
                                    #engineerUnit = None,
                                    integerDisplayFormat = 30,
                                    fractionDisplayFormat = 40
                                    )

        config.node.deviceList[0].analogTagList.append(analogTag)

    result = edgeAgent.uploadConfig(constant.ActionType['Create'], edgeConfig = config)
    

    # print('scada session #5')
    #print(output_df)
    edgeData = EdgeData()
    for feature in list(output_df):
        #edgeData = EdgeData()
        deviceID = device_id
        tagName = feature
        #print("===================================")
        #print(output_df[feature].tail(1).values[0])
        value = float(output_df[feature].tail(1).values[0])
        value = "%.2f" % round(value, 2)
        
#         print(value)
        tag = EdgeTag(deviceID,tagName,value)
        edgeData.tagList.append(tag)
        #print("===================================")
        #print(deviceID,tagName,value)
        #print("===================================")
        #print(edgeData)
        #print(len(str(value)))
        #print("===================================")
        #print(tag)
        # print('scada session #5-1')
        #result = edgeAgent.sendData(data=edgeData)
        # print('scada session #5-2')
        #time.sleep(1)

#     print(datetime.datetime.now(), '|',
#           inspect.getframeinfo(inspect.currentframe()).function, '| Scada send data')
    edgeData.timestamp = datetime.datetime.fromtimestamp(data_dict['timestamp']) - datetime.timedelta(hours=8)
    
#     print('edge datetime', datetime.datetime.fromtimestamp(data_dict['timestamp']) - datetime.timedelta(hours=8))
    
    print(datetime.datetime.now(), '|',
          inspect.getframeinfo(inspect.currentframe()).function, '| send data, edge datetime',
          datetime.datetime.fromtimestamp(data_dict['timestamp']) - datetime.timedelta(hours=8))
    
    result = edgeAgent.sendData(data=edgeData)

# for objects in bucket.list(prefix='tdms/'): 
# for objects in bucket.list(prefix='線一/'): 
    

In [31]:
# def get_rpms_from_FOMOS(signalid,
#                         db,
#                         start_date=None,
#                         end_date=None,
#                         tb_suffix='',
#                         ret_ids=[1,5,20],
#                         ret_names=['Acc RMS','Vel RMS','RPM']):
    
#     print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
    
#     #get rpm information from FOMOS DB, no timeout
#     #example: a = get_all_station_signals(server='10.20.246.188'); 
#     #         a = a=get_rpms_from_FOMOS(10,'PXI 111','2020-08-17','2020-08-17')
#     #         a = get_rpms_from_FOMOS(a['signalid'].iloc[61],a['db_name'].iloc[61],start_date='2020-08-19',end_date='2020-08-20')
# #     import pymssql
# #     import datetime
# #     import numpy as np
# #     import csv
# #     import pandas as pd
    
#     if start_date is None or end_date is None:
#         end_date = '{:%Y-%m-%d}'.format(datetime.datetime.today())
#         start_date = '{:%Y-%m-%d}'.format(datetime.datetime.today() - datetime.timedelta(days=1))  
        
# #     print('Data range: {}~{}'.format(start_date,end_date))
#     print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function, '| Data range:', 'from:', start_date, ' to', end_date)
    
    
#     conn_str = 'DRIVER={{SQL Server}};SERVER={};DATABASE={};UID={};PWD={};CHARSET=UTF8'.format(MSSQL_HOST,
#                                                                                                db,
#                                                                                                MSSQL_USER,
#                                                                                                MSSQL_PASS)
#     print('Connection string: {}'.format(conn_str))
#     print(datetime.datetime.now(), '| MSSQL')
    
#     conn = pymssql.connect(conn_str,unicode_results=True)
#     startdate = start_date + ' 00:00:00.000'
#     enddate = end_date + ' 23:59:59.999'    
#     sql = "select * from Rawdata_setting{} where signalid = {} and Data_time between '{}' and '{}' order by Data_time".format(tb_suffix,signalid,startdate,enddate)
# #     print('SQL command: {}'.format(sql))
#     print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function, '| MSSQL Statement:', sql)
    
#     data = pd.read_sql(sql,conn)    
#     conn.close()
#     feas = np.array(list(csv.reader(data['parameter_id_value'])))
    
#     if feas.shape[0]==0: return None
#     tmp_data = data['parameter_id_value'][0].split(',')
#     dataids = [i for i in range(1,len(tmp_data),2)]
#     feas = feas[:,dataids].astype(np.float)  
    
#     parids = [int(tmp_data[i]) for i in range(0,len(tmp_data),2)]
#     rev_ids=[parids.index(f) for i,f in enumerate(ret_ids)]
#     data.drop(columns=['parameter_id_value'],inplace=True)
#     return pd.concat([data,pd.DataFrame(feas[:,rev_ids],columns=ret_names)],axis=1)
    

In [32]:
def get_rpms_from_FOMOS1(signalid,
                         db,
                         start_date=None,
                         end_date=None,
                         tb_suffix='',
                         ret_ids=[1,5,20],
                         ret_names=['Acc RMS','Vel RMS','RPM'],
                         timeout=3):
    
    #get rpm information from FOMOS DB with timeout
    #example: a = get_all_station_signals(server='10.20.246.188'); 
    #         a = get_rpms_from_FOMOS1(10,'PXI 111','2020-08-17','2020-08-17')
    #         a = get_rpms_from_FOMOS1(a['signalid'].iloc[61],a['db_name'].iloc[61],start_date='2020-08-19',end_date='2020-08-20')

    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
    
    if start_date is None or end_date is None:
        end_date = '{:%Y-%m-%d}'.format(datetime.datetime.today())
        start_date = '{:%Y-%m-%d}'.format(datetime.datetime.today() - datetime.timedelta(days=1))  
        
#     print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function, '| Data range:', 'from:', start_date, ' to', end_date)
    conn_str = 'DRIVER={{SQL Server}};SERVER={};DATABASE={};UID={};PWD={};CHARSET=UTF8'.format(MSSQL_HOST,
                                                                                               db,
                                                                                               MSSQL_USER,
                                                                                               MSSQL_PASS)
#     print(datetime.datetime.now(), '| MSSQL connection:', conn_str)
    
    try:
        #conn = pyodbc.connect(conn_str,unicode_results=True)
        #conn = conn = func_timeout(timeout, pymssql.connect, args={conn_str},kwargs={'unicode_results':True})
        conn = pymssql.connect(server=MSSQL_HOST, 
                               user=MSSQL_USER, 
                               password=MSSQL_PASS, 
                               database=db)  
        
    except FunctionTimedOut:
        print ( 'Connection could not complete within {} seconds, hence terminated.'.format(timeout))
        return None
    except Exception as e:
        # Handle any exceptions that doit might raise here
        print(e)
        return None  
    
    startdate = start_date + ' 00:00:00.000'
    enddate = end_date + ' 23:59:59.999'    
    sql = "select * from Rawdata_setting{} where signalid = {} and Data_time between '{}' and '{}' order by Data_time".format(tb_suffix,signalid,startdate,enddate)

#     print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function, '| MSSQL Statement:', sql)
    
    try:
        #data = pd.read_sql(sql,conn)   
        data = func_timeout(timeout, pd.read_sql, args=(sql,conn))
    except FunctionTimedOut:
        print ( 'Query could not complete within {} seconds, hence terminated.'.format(timeout))
        return None
    except Exception as e:
        # Handle any exceptions that doit might raise here
        print(e)
        return None 
        
    conn.close()
    feas = np.array(list(csv.reader(data['parameter_id_value'])))
    if feas.shape[0]==0: return None
    tmp_data = data['parameter_id_value'][0].split(',')
    dataids = [i for i in range(1,len(tmp_data),2)]
    feas = feas[:,dataids].astype(np.float)  
    
    parids = [int(tmp_data[i]) for i in range(0,len(tmp_data),2)]
    rev_ids=[parids.index(f) for i,f in enumerate(ret_ids)]
    data.drop(columns=['parameter_id_value'],inplace=True)
    return pd.concat([data,pd.DataFrame(feas[:,rev_ids],columns=ret_names)],axis=1)
    

In [33]:
def extract_features (device_name,
                      prefix,
                      fs=-1,
                      device=None,
                      bearing=0,
                      temperature=0,
                      vibration=0,
                      opts=None):
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)

    edgeAgent = None
    if opts is not None:
#         smartboxID = opts['smartboxID']
#         ScadaID = opts['ScadaID']
#         scada_project = opts['scada_project']
#         scada_name = opts['scada_name']
#         scada_id = opts['scada_id']
#         device_name = opts['device_name']
#         device_id = opts['device_id'] 
#         if device is None: device = device_id

        date = opts['date']
        AllFOMOSStationSensorInfo = opts['AllFOMOSStationSensorInfo']
        FOMOSTsConstraint = opts['FOMOSTsConstraint']
        edgeAgent = opts['edgeAgent']
#         bWrite2Scada = opts['bWrite2Scada']        
#         bForceExtract = opts['bForceExtract']  
        rpms = opts['rpms']  
        
    bucket = s3_conn()
    objectss = [objects for objects in bucket.list(prefix=prefix)]
    
    #extract FOMOS RPM information
    if scada_name.lower()!='vpodpro' and AllFOMOSStationSensorInfo is not None and rpms is None:        
        srollerID = re.findall('.*#([0-9]{1,3}).*',device_name)[0]
        find, frow = [(find,frow) for find,frow in AllFOMOSStationSensorInfo.iterrows() if \
                    srollerID in frow['signal_name'] and 'WS' in frow['signal_name'] and \
                    'Roller' in frow['signal_name']][0]
        
        fsignalid = frow['signalid']
        fdb = frow['db_name']  
        start_date = date.strftime('%Y-%m-%d')
        end_date = start_date   
        
        print(fsignalid,fdb,start_date,end_date, 'get_rpm')
        
        rpms = get_rpms_from_FOMOS1(fsignalid,fdb,start_date,end_date)
        if rpms is not None:
            rpms['timestamp'] = [int(ts.to_pydatetime().timestamp()) for ts in rpms['Data_time']]
        else:
            rpms = pd.DataFrame([[-9999,0]],columns=['timestamp','RPM'])
            print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function, '| Retrieve FOMOS RPM information fails, adopt a fake one.')
        pass
    
    if scada_name.lower()!='vpodpro' and AllFOMOSStationSensorInfo is None and bForceExtract:
        rpms = pd.DataFrame([[-9999,0]],columns=['timestamp','RPM'])
        print('******{} Adopt fake FOMOS RPM information.'.format(\
        datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S.%f'))) 
        
    for objects in objectss: 
        # translate Chinese etc. into url 
        # The URL quoting functions focus on taking program data and making it safe for use as URL components by quoting special characters and appropriately encoding non-ASCII text.
        objs = urllib.parse.quote(objects.key)
        bExtractSave = True
        #two kinds of files: TDMS & db, to consider
        #if ((scada_project == '條二' or scada_project == '線二') and ('Raw' in objs and 'index' not in objs)) or            ((scada_project == '線一') and ('Raw' in objs and 'index' not in objs and 'rolling' in objs)):
        if 'Raw' in objs and 'index' not in objs and scada_name.lower()!='vpodpro':
            # extract filename only
            filename = objs.split('/')[-1]
            #Key: #1HSM/ROT/TDMS/#10內冷式ROT Roller WS/2020/02/24/Raw Data-#10內冷式ROT Roller WS-00-09-57.tdms
            timestamp = retrieve_ts(objs, filename)
            new_prefix = '/'.join(objs.split('/')[:-1]) + '/'
            #print(timestamp)
            #there should be error handling procedure  
            #apply FOMOS timestamp Constraint
            #print(rpms)
            ts_diff = np.abs(rpms['timestamp'].values-timestamp)
            wh_min_diff = np.argmin(ts_diff)
            
            #print(datetime.datetime.now(), '|',
            #      inspect.getframeinfo(inspect.currentframe()).function, '|  extracted:',
            #      int(timestamp),
            #      rpms['timestamp'].values[wh_min_diff],
            #      ts_diff[wh_min_diff],
            #      FOMOSTsConstraint)
            
            if ts_diff[wh_min_diff]>FOMOSTsConstraint: 
                #print(datetime.datetime.now(), '|',
                #      inspect.getframeinfo(inspect.currentframe()).function,
                #      '|  larger than FOMOSTsConstraint, bExtractSave=False, bypass')
                bExtractSave = False
                this_rpm = 0
                data_dict = {}     
            else:
                print(datetime.datetime.now(), '|',
                      inspect.getframeinfo(inspect.currentframe()).function,
                      '|  larger than FOMOSTsConstraint, bExtractSave=True')
                this_rpm = rpms['RPM'].iloc[wh_min_diff] 
                bExtractSave = True
                
            bExtractSave = bExtractSave or bForceExtract
#             print(datetime.datetime.now(), '|',
#                       inspect.getframeinfo(inspect.currentframe()).function, '|  bExtractSave:', bExtractSave)
            
            
            
            #if bExtractSave: 
                
            data_dict = query_features(filename,
                                           prefix if filename in prefix else new_prefix, 
                                           scada_name)
            data_dict['RPM'] = this_rpm
                # vProdPro features
                #data_dict['vibration'] = data_dict['Velocity RMS band 0']  
            vibration = data_dict['Velocity RMS band 0']  
 
        elif scada_name.lower()=='vpodpro':#not finished yet!
            filename = objs.split('/')[-1]            
            #Key: Key: #1HSM/ROT/vPodPRO/#1內冷式ROT Roller WS_vpod/2019/11/14/Raw Data-#1內冷式ROT Roller WS_vpod-18-24-43_25600.bin
            timestamp = retrieve_ts(objs, filename,type='bin')
            if fs==-1: 
                tmp = filename.split('_')[-1].split('.')
                if len(tmp)<2: fs = 25600 
                fs = 25600 if not tmp[0].isnumeric() else int(tmp[0])
            #there should be error handling procedure    
            data_dict = query_features(filename, prefix, scada_name, fs=fs)
            data_dict['RPM'] = 0
        else:
            bExtractSave = False
            
                
        data_dict['bearing'] = bearing
        data_dict['temperature'] = temperature
        data_dict['vibration'] = vibration
        data_dict['type'] = 'unloading'
        data_dict['device'] = device
        data_dict['timestamp'] = int(timestamp)
        data_dict['scada'] = scada_name            
                 
            
        if bExtractSave:
            
            
            mgdb_device_df = mgdb_df[mgdb_df['device']==device]
            mgdb_ts_df = mgdb_device_df[mgdb_device_df['timestamp'] == int(data_dict['timestamp'])]
            
            if len(mgdb_ts_df)==0:
                
                print(datetime.datetime.now(), '|',
                      inspect.getframeinfo(inspect.currentframe()).function,
                      '|  no data in MongoDB')
                
                
                insert_mgdb(data_dict)        
                scada_client(data_dict)
                time.sleep(3)
                
        #if edgeAgent is not None and bWrite2Scada and bExtractSave: scada_client(data_dict)
        #if bExtractSave and bWrite2Scada: scada_client(data_dict)
        #if bExtractSave: time.sleep(3)

In [34]:
# def get_prefix_list (scada_project,bucket):
    
#     print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
    
#     prefix_list = []
#     for key in bucket.list(delimiter='/', prefix=scada_project):
        
#         if key.name.count('/')!=1:

#             for j in bucket.list(delimiter='/', prefix=key.name):
#                 if j.name.count('/')!=2:
#                     prefix_list.append(j.name)
                     
#     return prefix_list

In [35]:
# def feature_extraction(date):
    
#     print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)

#     # combine into S3 key
#     if scada_name.lower()=='vpodpro':#not finished yet!
#         prefix_ = scada_project + '/ROT/vProdPRO/RAW_DB/'
#         retrieve_bin_from_dbs ()
#     else:
#         prefix_ = scada_project + '/ROT/TDMS/' + device_name + '/'        
#         #should single_date -> date ??
#         year = "{:04d}".format(date.year)
#         month = "{:02d}".format(date.month)
#         day = "{:02d}".format(date.day)
        
#         # variable 'extract_features' specifies which day begining to extract features.
#         extraction_anchor = prefix_ + str(year) + '/' + str(month) + '/' + str(day) + '/'
#         extract_features (device_name, 
#                           extraction_anchor,
#                           device=device_id)

In [36]:
def get_signal_names (StationName='111'):  
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
#     import pandas as pd
#     import pymssql
    #example: GetSignalNames(StationName='112',server='10.20.246.145')
    StationNames = get_station_names()
    
    if not (StationName in StationNames['client_name'].tolist()): 
        print('Station {} is not in {}'.format(StationName,server))
        return None
    
    db = StationNames[StationNames['client_name']==StationName]['db_name'].iloc[0]
    conn = pymssql.connect(server=MSSQL_HOST, 
                           user=MSSQL_USER, 
                           password=MSSQL_PASS, 
                           database=db)   
    
    sql = 'select * from signal_ID'
    data = pd.read_sql(sql,conn)
    conn.close()
    return data
   

In [37]:
 
def get_all_station_signals ():
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
#     import pandas as pd
    
    #example: signals=get_all_station_signals(); signals.index=range(signals.shape[0]); signals.to_csv('signals.csv',encoding='utf-8-sig')
    StationNames = get_station_names()
    ress = []
    for i in range(StationNames.shape[0]):
        StationName = StationNames['client_name'].iloc[i]
        Info = StationNames.iloc[i,:]
        SignalNames = get_signal_names (StationName=StationName)[['signalid','signal_name','channel_ID']]
        res = pd.concat([pd.DataFrame(Info.values.reshape(1,-1).repeat(SignalNames.shape[0],axis=0),columns=Info.index.tolist()),SignalNames],axis=1)
        ress.append(res)
    result = pd.concat(ress)
    result.index=range(result.shape[0])
    return result
    

In [38]:
def get_station_names():
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
#     import pymssql
#     import pandas as pd 
    
    #example: get_station_names(server='10.20.246.145')
    #conn = pymssql.connect('DRIVER={{SQL Server}};SERVER={};DATABASE={};UID={};PWD={}'.format(server,db,uid,pwd))
    conn = pymssql.connect(server=MSSQL_HOST, 
                           user=MSSQL_USER,
                           password=MSSQL_PASS, 
                           database=MSSQL_DB)  
    
    sql = 'select * from client_info'
    data = pd.read_sql(sql,conn)
    conn.close()
    
    return data
    

In [39]:
def feature_extraction2 (date,opts):
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
    
#     smartboxID = opts['smartboxID']
#     ScadaID = opts['ScadaID']
#     scada_project = opts['scada_project']
#     scada_name = opts['scada_name']
#     scada_id = opts['scada_id']
#     device_name = opts['device_name']
#     device_id = opts['device_id']
    opts['date'] = date
    #opts['AllFOMOSStationSensorInfo'] = None
    
    # combine into S3 key
    if scada_name.lower()=='vpodpro':#not finished yet!
        prefix_ = '#1HSM/ROT/vProdPRO/RAW_DB/'
        retrieve_bin_from_dbs (opts=opts)
    else:
        if opts['AllFOMOSStationSensorInfo'] is None:
#             print('******{} retrieve FOMOS sensor list.'.format(\
#             datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S.%f'))) 
            
            print(datetime.datetime.now(), '| Retrieve FOMOS sensor list')
            
            AllFOMOSStationSensorInfo = get_all_station_signals()
            opts['AllFOMOSStationSensorInfo'] = AllFOMOSStationSensorInfo
            
        prefix_ = '#1HSM/ROT/TDMS/' + device_name + '/'        
        #should single_date -> date ??
        year = "{:04d}".format(date.year)
        month = "{:02d}".format(date.month)
        day = "{:02d}".format(date.day)
        
        # variable 'extract_features' specifies which day begining to extract features.
        prefix = prefix_ + str(year) + '/' + str(month) + '/' + str(day) + '/'
        
        extract_features(device_name,
                         prefix,
                         device=device_id,
                         opts=opts)

In [40]:
def scada_conn ():
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
    
    options = EdgeAgentOptions(
        reconnectInterval = 1, # MQTT reconnect interval seconds
        nodeId = scada_id, # Getting from SCADA portal
        deviceId = device_id, # If type is Device, DeviceId must be filled
        type = constant.EdgeType['Gateway'], # Choice your edge is Gateway or Device, Default is Gateway
        heartbeat = 60, # Default is 60 seconds
        dataRecover = False, # Need to recover data or not when disconnected
        connectType = constant.ConnectType['MQTT'], # Connection type (DCCS, MQTT), default is DCCS
        
        MQTT = MQTTOptions( # If connectType is MQTT, must fill this options
            hostName = DHB_MQTT_HOST,
            port = DHB_MQTT_PORT,
            userName = DHB_MQTT_USER,
            password = DHB_MQTT_PASS,
            protocalType = constant.Protocol['TCP'] # MQTT protocal (TCP, Websocket), default is TCP
        )
    )

    edgeAgent = EdgeAgent( options = options )
    edgeAgent.connect()
    
    return edgeAgent
    

In [41]:
def s3_conn():
    
    #print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)

    s3_connection = boto.connect_s3(
                       aws_access_key_id = ACCESS_KEY,
                       aws_secret_access_key = SECRET_KEY,
                       host = HOST,
                       port = PORT,
                       is_secure=False,               # uncomment if you are not using ssl
                       calling_format = boto.s3.connection.OrdinaryCallingFormat(),
                     )
    bucket = s3_connection.get_bucket(BUCKET_NAME, validate=False)
    
    return bucket
    

In [42]:
def extract_feature_to_wisepaas(
                                FOMOSTsConstraint=120,
                                FOMOSSensorInfo=None):

    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function)
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function, '| scada_project:', scada_project)
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function, '| scada_name:', scada_name)
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function, '| device_id:', device_id)
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function, '| scada_id:', scada_id)
    
    day_count = (datetime.datetime.strptime(os.environ['to'], '%Y-%m-%d') -
                 datetime.datetime.strptime(os.environ['from'], '%Y-%m-%d')).days
    today = (datetime.datetime.now().date()).strftime('%Y-%m-%d')
    
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function, '| dt_from:', os.environ['from'])
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function, '| dt_to:', os.environ['to'])
    print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function, '| dt_now:', today)


    
    edgeAgent = None
    if bWrite2Scada:
        edgeAgent = scada_conn ()
        
    opts = {'FOMOSTsConstraint':FOMOSTsConstraint,
            'edgeAgent':edgeAgent,
            'AllFOMOSStationSensorInfo':FOMOSSensorInfo,
            'rpms':None
           }
#     print(opts)
#     opts['tag_list_pd'] = sensor_list

    mgdb_date_list = []
    if not bForceExtract:
        
        print(datetime.datetime.now(), '|',
              inspect.getframeinfo(inspect.currentframe()).function, '| disable bForceExtract')
        
        ## query mongodb
        #temp = query_mgdb(os.environ['from'], 
        #                  os.environ['to'], 
        #                  device_id = device_id)
        
        temp = mgdb_df[mgdb_df['device']==device_id]
    
        print(datetime.datetime.now(), '|',
              inspect.getframeinfo(inspect.currentframe()).function, '| Number of datapoints in mongodb:', len(temp))
    
        if temp.shape[0]>0:
            mgdb_date_list = pd.to_datetime(temp.timestamp, unit='s').dt.strftime('%Y-%m-%d').unique().tolist()
        ## query mongodb end

    
    
    s3_date_list = []
    bucket = s3_conn()
    
    
    if scada_name.lower()=='vpodpro':
        #prefix = scada_project + '/ROT/vPodPRO/RAW_DB/' + device_name + '/' + str(year) + '/' + str(month) + '/' + str(day) + '/'
        feature_extraction2('',opts)
        pass
    else:
#         for single_date in (datetime.datetime.now().date() - datetime.timedelta(n) for n in range(day_count)):
        for single_date in (datetime.datetime.strptime(os.environ['to'], '%Y-%m-%d') - datetime.timedelta(n) for n in range(day_count)):

            ## query s3
            year = "{:04d}".format(single_date.year)
            month = "{:02d}".format(single_date.month)
            day = "{:02d}".format(single_date.day)

            # combine into S3 key
            prefix = '#1HSM/ROT/TDMS/' + device_name + '/' + str(year) + '/' + str(month) + '/' + str(day) + '/'
            
            #print(datetime.datetime.now(), '|',
            #      inspect.getframeinfo(inspect.currentframe()).function,
            #      '|  s3_prefix=', prefix)
            
            b_list = bucket.list(prefix=prefix)

            if len(list(b_list)) > 0:
                s3_date_list.append(single_date.strftime('%Y-%m-%d'))
            ## query s3 end

            
        ## compare mongodb and s3
        date_diff = list(set(s3_date_list) - set(mgdb_date_list))
        date_diff = sorted(date_diff)
        print(datetime.datetime.now(), '|', inspect.getframeinfo(inspect.currentframe()).function,
              '| date_diff, mongodb list and s3 list=', len(date_diff), len(mgdb_date_list), len(s3_date_list))
        ## compare mongodb and s3 end
            
        
        if FOMOSSensorInfo is None:
            print(datetime.datetime.now(), '|',
                  inspect.getframeinfo(inspect.currentframe()).function, 
                  '| Retrieve FOMOS sensor list')
        
            FOMOSSensorInfo = get_all_station_signals()
            opts['AllFOMOSStationSensorInfo'] = FOMOSSensorInfo
        
        #extract rpms information
        if FOMOSSensorInfo is not None:
            srollerID = re.findall('.*#([0-9]{1,3}).*',device_name)[0]
            find, frow = [(find,frow) for find,frow in FOMOSSensorInfo.iterrows() if \
                        srollerID in frow['signal_name'] and 'WS' in frow['signal_name'] and \
                        'Roller' in frow['signal_name']][0]
            
            fsignalid = frow['signalid']
            fdb = frow['db_name'] 
            
            rpms = get_rpms_from_FOMOS1(fsignalid,
                                        fdb,
                                        os.environ['to'],
                                        today)
            
            if rpms is not None:
                rpms['timestamp'] = [int(ts.to_pydatetime().timestamp()) for ts in rpms['Data_time']] 
                
            result = 'failed' if rpms is None else 'successfully'
            print(datetime.datetime.now(), '|',
                  inspect.getframeinfo(inspect.currentframe()).function, '| Retrieve FOMOS RPM information', result)
            
            opts['rpms'] = rpms
            
            
        ## extraction
        for single_date in date_diff:

            print(datetime.datetime.now(), '|',
                  inspect.getframeinfo(inspect.currentframe()).function,
                  '| extract features date:', single_date)
            
            feature_extraction2(datetime.datetime.strptime(single_date, "%Y-%m-%d"), opts)  
            
            
    if edgeAgent is not None: edgeAgent.disconnect()
    pass

In [ ]:

mgdb_df = query_mgdb_dt(os.environ['from'], os.environ['to'])

for dname, did in zip(device_name_list, device_id_list):
    
    device_id=did
    device_name=dname
    
    extract_feature_to_wisepaas(FOMOSTsConstraint=70)
    
    print(datetime.datetime.now(), '|*')

2020-11-12 13:50:57.800948 | query_mgdb_dt
2020-11-12 13:51:03.425788 | more than one points in MongoDB
2020-11-12 13:51:03.428114 | extract_feature_to_wisepaas
2020-11-12 13:51:03.435090 | extract_feature_to_wisepaas | scada_project: Y4_FOMOS_1HSM
2020-11-12 13:51:03.439867 | extract_feature_to_wisepaas | scada_name: 111
2020-11-12 13:51:03.444556 | extract_feature_to_wisepaas | device_id: Y4121110101
2020-11-12 13:51:03.449427 | extract_feature_to_wisepaas | scada_id: 14d267b3-b03c-48d3-9b54-5ca79d0c457b
2020-11-12 13:51:03.454793 | extract_feature_to_wisepaas | dt_from: 2020-09-01
2020-11-12 13:51:03.459199 | extract_feature_to_wisepaas | dt_to: 2020-11-12
2020-11-12 13:51:03.463501 | extract_feature_to_wisepaas | dt_now: 2020-11-12
2020-11-12 13:51:03.467653 | scada_conn
2020-11-12 13:51:03.484671 | extract_feature_to_wisepaas | disable bForceExtract
2020-11-12 13:51:03.500006 | extract_feature_to_wisepaas | Number of datapoints in mongodb: 0
Connected OK Returned code= 0
subscribe

In [ ]:
# TODO1: (done) consistency function name
# TODO2: precise comparision between S3 and MongoDB
# TODO3: variable scada_project should be consistency
# TODO4: remove opt
# TODO5: split extract_features for TDMS and BIN
# TODO6: specify datetime for vPodPRO
# TODO7: refact flags
